# **Summarisation & Evasion Notebook (LB)**

# **1. Objective**

- Summarise banker answers into short, PRA relevant insights.
- Generate an evasion score to tag summaries.
- RAG pipeline to bring in relevant external documents (e.g. PRA risk definitions, regulatory news).
- Optional extension: validate flagged risks with external/regulatory news.

# **2. Set up Workspace**

In [1]:
# Import libraries
# Core python
import os
import numpy as np
import pandas as pd
import re
import json
import pathlib
from pathlib import Path
from typing import List, Dict, Any 
import csv

# NLP & Summarisation
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import spacy
from llama_cpp import Llama 

# Evaluation
from rouge_score import rouge_scorer
import evaluate
from bert_score import score as bertscore 

# Retrieval
from sentence_transformers import SentenceTransformer 
import faiss
import chromadb
import langchain
import llama_index

# ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns 

/opt/homebrew/anaconda3/envs/nlp-evasion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **3. Load the dataset**

In [2]:
# Load the dataset.
jpm_2025_df = pd.read_csv('../data/processed/jpm/all_jpm_2025.csv')

# View the data.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...


# **4. Preprocessing**

In [3]:
# View speaker roles.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer',
       'And then some. Theres a lot of value added.', 'Okay'],
      dtype=object)

In [4]:
# View rows with invalid roles.
valid_roles = 'analyst', 'Chief Financial Officer', 'Chairman & Chief Executive Officer'
invalid_roles_df = jpm_2025_df[~jpm_2025_df['role'].isin(valid_roles)]

# Number of rows with invalid roles.
print('Number of rows:', invalid_roles_df.shape[0])

# View the rows.
invalid_roles_df.head()

Number of rows: 2


,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
201,35,5.0,"Chief Financial Officer, JPMorganChase",And then some. Theres a lot of value added.,JPMorganChase,"Yeah. And obviously, I mean, we're not going t...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...
205,36,3.0,"Chief Financial Officer, JPMorganChase",Okay,there you have it.,"But it's not like I thought it would do badly,...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...


In [5]:
# Input the correct role information.
jpm_2025_df.at[205, 'role'] = 'Chief Financial Officer'
jpm_2025_df.at[209, 'role'] = 'Chief Financial Officer'

# Verify the roles have been updated.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer',
       'And then some. Theres a lot of value added.'], dtype=object)

In [6]:
# Define role mapping.
role_map = {
    'analyst': 'analyst',
    'Chief Financial Officer': 'banker',
    'Chairman & Chief Executive Officer': 'banker'
}

# Apply to dataset.
jpm_2025_df['role_normalised'] = jpm_2025_df['role'].map(role_map)

In [7]:
# View the dataset.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf,role_normalised
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker


# **5. Summarisation**

## **5.1 Baseline**
- Model = BART (not instruction tuned)

In [8]:
# Filter data to banker answers only.
banker_answers = jpm_2025_df[jpm_2025_df['role_normalised'] == 'banker']['content'].tolist()
print(banker_answers[0][:200])

Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the w


In [9]:
# Summarisation baseline (BART)
summariser = pipeline('summarization', model='facebook/bart-large-cnn')

sample_text = banker_answers[0]
summary = summariser(sample_text, max_length=80, min_length=30, do_sample=False)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Device set to use mps:0


Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: The main thing that we see there, what would appear to be a certain amount of frontloading of spending ahead of people expecting price increases from tariffs. So ironically, that's actually somewhat supportive, all else equal. In terms of our corporate clients, obviously, they've been reacting to the changes in tariff policy.


In [10]:
# Prompt conditioning to make PRA relevant.
prompt = "Summarise this answer, focusing on risk, capital and evasion of detail: " + sample_text
summary = summariser(prompt, max_length=80, min_length=30)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: Corporates are taking a wait-and-see approach to tariff policy. Some sectors are going to be much more exposed than others. Small business and smaller corporates are probably a little more challenged.


## **5.2 Adding Context**

Retrieve PRA risk categories to give greater PRA focus to summaries (local RAG loop).
- measure cosine similarity between transcript chunks and PRA risk categories (vectors)
- retrieve the top 2-3 most relevant risk categories 
- prepend them to the summarisation prompt to make summaries PRA-aligned instead of just summarised answers

- Attempting to use BART resulted in prompt echoing.
- New attempt using Mistral-7B-Instruct.
- Using sentence-BERT vs TF-IDF for vectorisation.

In [11]:
# Function to remove whitespace in text.
def clean_text(text: str):
    return re.sub(r'\s+', ' ', text).strip()

In [12]:
# Function to split the transcript into smaller chunks.
def chunk_text(text: str, max_chars: int = 6000):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip()) # split into sentences 
    chunks, current_chunk, current_len = [], [], 0 # list of chunks, sentences collecting for current chunk, character count for current chunk

    for s in sentences:
        if current_len + len(s) + 1 <= max_chars: # if the characters of current chunk + new sentence is below the limit:
            current_chunk.append(s) # add sentence to current chunk 
            current_len += len(s) + 1 # update running character count 
        
        else: # if the characters is above the limit:
            chunks.append(' '.join(current_chunk)) # add the current chunk to the final chunk list
            current_chunk, current_len = [s], len(s) # start a new chunk containing the sentence and update current len

    if current_chunk:
        chunks.append(' '.join(current_chunk)) # add any sentences in current chunk after loop ends 

    return chunks 

In [ ]:
# Function to load PRA categories and definitions from CSV.
def load_pra_categories(path: Path):
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        return [
            (row.get('category', '').strip(), [row.get('definition', '').strip()])
            for row in reader if row.get('category')
        ]

In [ ]:
# Build a Sentence-BERT embedding index for PRA categories.
def build_embedding_index(pra_categories):
    embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    docs = [f"{name} {' '.join(defs)}" for name, defs in pra_categories]
    pra_risk_embeddings = embedder.encode(docs, batch_size=32, normalize_embeddings=True)

    return embedder, np.asarray(pra_risk_embeddings)

In [15]:
# Function to find the relevant PRA categories to the transcript chunks.
def find_rel_categories(chunk, pra_categories, embedder, pra_risk_embeddings, top_k=2):
    query_vec = embedder.encode([chunk], normalize_embeddings=True) # turns chunk into embedding
    sims = cosine_similarity(query_vec, pra_risk_embeddings).ravel() # compares the chunk to each category doc 
    top_indices = np.argsort(-sims)[:top_k] # sorts scores descending and selected top k cateogories 

    return [pra_categories[i] for i in top_indices]

In [16]:
# def extract_json_block(model_output: str) -> str:
#     """If the model adds extra text, grab the first JSON object."""
#     a = model_output.find("{")
#     b = model_output.rfind("}")
#     return model_output[a:b+1] if a != -1 and b != -1 and b > a else model_output

In [17]:
# JSON instructions for the output file.
JSON_INSTRUCTIONS = """\
You are a careful data extraction model.
Return ONLY valid minified JSON with EXACT keys: summary, evidence, pra_categories.
Schema:
{
  "summary": "string (4-6 sentences, neutral tone)",
  "evidence": ["string", ...],               // up to N quotes/facts
  "pra_categories": [                        // 1-3 items
    {"name": "string", "why": "string"},
    ...
  ]
}
Rules:
- Use standard double quotes; no trailing commas; no markdown or prose.
- If something is unknown, return an empty string or [] (do NOT omit keys).
"""

In [18]:
# [PLACEHOLDER]

def _repair_json_string(s: str) -> str:
    if not isinstance(s, str):
        return ""
    t = s.strip().replace("\r\n", "\n").lstrip("\ufeff")

    # Prefer <json>...</json> blocks if present
    m = re.search(r"<json>\s*(\{[\s\S]*?\})\s*</json>", t, flags=re.IGNORECASE)
    if m:
        t = m.group(1).strip()
    else:
        # If fenced code exists, prefer ```json ... ``` then ``` ... ```
        m = re.search(r"```json\s*([\s\S]*?)```", t, flags=re.IGNORECASE)
        if not m:
            m = re.search(r"```\s*([\s\S]*?)```", t)
        if m:
            t = m.group(1).strip()

        # As a last resort, grab the *first* minimal JSON object
        if not m:
            m = re.search(r"\{[\s\S]*?\}", t)
            if m: t = m.group(0).strip()

    # Normalise quotes and remove trailing commas before } or ]
    t = t.replace("“", '"').replace("”", '"').replace("’", "'")
    t = re.sub(r",\s*([}\]])", r"\1", t)
    return t

def _coerce_schema(obj: dict, max_evidence: int) -> dict:
    """Ensure keys exist and types are consistent."""
    out = {
        "summary": obj.get("summary") or "",
        "evidence": obj.get("evidence") or [],
        # accept both pra_categories and "pra categories"
        "pra_categories": obj.get("pra_categories") or obj.get("pra categories") or [],
    }

    # evidence -> list[str], capped
    if not isinstance(out["evidence"], list):
        out["evidence"] = [str(out["evidence"])]
    out["evidence"] = [str(x).strip() for x in out["evidence"] if str(x).strip()][:max_evidence]

    # pra_categories -> list[{name, why}]
    pcs = []
    for c in out["pra_categories"]:
        if isinstance(c, dict):
            pcs.append({"name": str(c.get("name") or ""), "why": str(c.get("why") or "")})
        else:
            pcs.append({"name": str(c), "why": ""})
    out["pra_categories"] = pcs
    return out

def _parse_llm_json(text: str, max_evidence: int) -> dict:
    t = _repair_json_string(text)
    try:
        obj = json.loads(t)
        return _coerce_schema(obj, max_evidence)
    except Exception:
        # Never return raw; return a well-formed empty structure
        return _coerce_schema({}, max_evidence)

In [19]:
# [PLACEHOLDER]
def summarise_chunk(model, chunk, relevant_categories, chunk_idx, max_evidence=5):
    # Build PRA notes block
    lines = []
    for name, definition in relevant_categories:
        lines.append(f"- {name}:")
        for d in list(definition)[:2]:
            lines.append(f"- {d}")
    notes_block = "\n".join(lines)

    system_prompt = (
        JSON_INSTRUCTIONS
        + "\nYou are a Data Scientist producing PRA-aligned summaries. "
          "Use only the transcript text as evidence. Map content to the PRA categories provided."
    )

    user_prompt = f"""
TRANSCRIPT:
{clean_text(chunk)}

PRA NOTES:
{clean_text(notes_block)}

TASK:
Return JSON ONLY, wrapped exactly like this:
<json>{"summary": "...", "evidence": ["..."], "pra_categories": [{"name":"...","why":"..."}]}</json>

RULES:
- 4-6 sentence neutral summary.
- Up to {max_evidence} evidence bullets (quotes/facts).
- 1-3 pra_categories objects.
- If evidence is lacking, use a single bullet "Insufficient evidence".
- Only choose categories supported by the evidence.
""".strip()

    response = model.create_chat_completion(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": clean_text(user_prompt)},
        ],
        temperature=0.2,
        top_p=0.9,
        max_tokens=700,   # keep headroom to avoid truncation
        repeat_penalty=1.1,
    )

    text = response["choices"][0]["message"]["content"].strip()
    return _parse_llm_json(text, max_evidence, debug_prefix=f'chunk_{chunk_idx}')

In [20]:
# # Function to summarise the chunk.
# def summarise_chunk(model, chunk, relevant_categories, max_evidence=5):
#     lines = []
#     for name, definition in relevant_categories:
#         lines.append(f'- {name}:')
#         for d in definition:
#             lines.append(f'- {d}')
#     notes_block = '\n'.join(lines)

#     system_prompt = (
#         JSON_INSTRUCTIONS
#         + '\nYou are a Data Scientist producing PRA-aligned summaries.'
#         'Use only the transcript text as evidence. Map content to the PRA categories provided.'
#     )

#     user_prompt = f"""
# TRANSCRIPT:
# {chunk}

# PRA NOTES:
# {notes_block}

# TASK:
# Return STRICT JSON with keys:
# - evidence: list of up to {max_evidence} bullet strings
# - pra_categories: list of objects {{ 'name': '...', 'why': '...' }}
# - summary: 4-6 sentences, neutral tone

# RULES:
# - Do not invent facts. If evidence is lacking, use a single bullet 'Insufficient evidence'.
# - Only choose categories supported by the evidence.
# - No markdown. Output JSON only.
# """
    
#     response = model.create_chat_completion(
#         messages=[
#             {'role': 'system', 'content': system_prompt},
#             {'role': 'user', 'content': clean_text(user_prompt)}
#         ],
#         temperature=0.2,
#         max_tokens=700,
#         repeat_penalty=1.1
#     ) 

#     text = response['choices'][0]['message']['content'].strip()
#     try:
#         return json.loads(extract_json_block(text))
#     except Exception:
#         return {'raw': text}

In [21]:
# Define variables.
MODEL_PATH = '/Users/laurenbrixey/Documents/Data Science Career Accelerator/Project Submissions/Course 3/topic_project_4.1/mistral-7b-instruct-v0.1.Q4_K_M.gguf'
PRA_NOTES_PATH = '../data/RAG-resources/PRA_risk_categories.csv'
TRANSCRIPT_PATH = '../data/processed/jpm/all_jpm_2025.csv'
OUTPUT_PATH = "jpm_mistral_pra_summary.json"
TOP_K = 2

In [22]:
# Runner code.
pra_categories = load_pra_categories(Path(PRA_NOTES_PATH)) # load pra category file
embedder, category_embeddings = build_embedding_index(pra_categories) # embed pra categories

transcript_text = Path(TRANSCRIPT_PATH).read_text() # load the transcript text
transcript_chunks = chunk_text(transcript_text) # split the transcript into smaller chunks

n_threads = max(4, (os.cpu_count() or 8) - 2)

model = Llama(
    model_path=str(MODEL_PATH), 
    n_ctx=4096, 
    n_gpu_layers=24, 
    chat_format='mistral-instruct', 
    n_threads=n_threads)

results = []
for i, chunk in enumerate(transcript_chunks, 1):
    # top_categories = find_rel_categories(chunk, pra_categories, embedder, category_embeddings, top_k=TOP_K)
    # summary_result = summarise_chunk(model, chunk, top_categories)
    # results.append({'chunk': i, 'result': summary_result})
    try:
        # Use your intended TOP_K here
        top_categories = find_rel_categories(
            chunk, pra_categories, embedder, category_embeddings, top_k=TOP_K
        )

        summary_result = summarise_chunk(model, chunk, top_categories, i, max_evidence=5)

        results.append({'chunk': i, 'result': summary_result})

    except Exception as e:
        # Hard fail-safe: still emit a valid empty structure for this chunk
        results.append({
            'chunk': i,
            'result': {
                'summary': '',
                'evidence': ['Insufficient evidence'],
                'pra_categories': []
            },
            'error': str(e)
        })

combined_summary = ' '.join(r['result'].get('summary', '') for r in results)
final_output = {'chunks': results, 'combined_summary': combined_summary.strip()}

Path(OUTPUT_PATH).write_text(json.dumps(final_output, indent=2, ensure_ascii=False))

NameError: name 'pra_risk_df' is not defined

In [ ]:
# # Post processing script.
# INPUT_JSON = '../notebooks/jpm_mistral_pra_summary.json' # input path
# INPUT_PATH = pathlib.Path(INPUT_JSON)

# # Output paths 
# OUTPUT_JSON = INPUT_PATH.with_suffix('.clean.json')
# OUTPUT_CSV = INPUT_PATH.with_suffix('.csv')

# # Load the raw file.
# with open(INPUT_PATH, 'r', encoding='utf-8') as f:
#     raw_file = json.load(f)

# chunks = raw_file.get('chunks', []) # get the list of chunks

# rows = [] # empty list to store cleaned rows

# for row in chunks:
#     result= row.get('result', {})

#     # Handle inconsistent key names.
#     pra_categories = result.get('pra_categories') or result.get('pra categories')
#     evidence = result.get('evidence') or result.get('raw')

#     rows.append({
#         'chunk_id': row.get('chunk'),
#         'summary': result.get('summary'), 
#         'pra_categories': pra_categories,
#         'supporting_evidence': evidence
#     })

# # Save clean JSON.
# with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
#     json.dump(rows, f, ensure_ascii=False, indent=2)

# # Save clean csv
# df = pd.DataFrame(rows)
# df.to_csv(OUTPUT_CSV, index=False)

# print("Clean JSON saved to:", OUTPUT_JSON)
# print("CSV saved to:", OUTPUT_CSV)

In [ ]:
# INPUT_JSON = pathlib.Path('../notebooks/jpm_mistral_pra_summary.json')
# OUTPUT_JSON = INPUT_JSON.with_suffix(".clean.v2.json")
# OUTPUT_CSV  = INPUT_JSON.with_suffix(".v2.csv")

# def safe_loads(s: str):
#     try:
#         return json.loads(s)
#     except Exception:
#         return None

# def pick(d, *keys):
#     if not isinstance(d, dict): return None
#     for k in keys:
#         v = d.get(k)
#         if v is not None:
#             return v
#     return None

# raw = json.load(open(INPUT_JSON, "r", encoding="utf-8"))
# rows = []
# for ch in raw.get("chunks", []):
#     res = ch.get("result", {}) or {}
#     raw_parsed = safe_loads(res.get("raw")) if isinstance(res.get("raw"), str) else None

#     summary = pick(res, "summary") or pick(raw_parsed or {}, "summary")
#     pra_categories = (
#         pick(res, "pra_categories", "pra categories")
#         or pick(raw_parsed or {}, "pra_categories", "pra categories")
#     )
#     evidence = pick(res, "evidence") or pick(raw_parsed or {}, "evidence")

#     rows.append({
#         "chunk_id": ch.get("chunk"),
#         "summary": summary,
#         "pra_categories": pra_categories,
#         "supporting_evidence": evidence,
#     })

# # Save files
# with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
#     json.dump(rows, f, ensure_ascii=False, indent=2)

# pd.DataFrame(rows).to_csv(OUTPUT_CSV, index=False)
# print("Wrote:", OUTPUT_JSON)
# print("Wrote:", OUTPUT_CSV)

# **6. Evasion Scoring**